## Example 1: Discovering HRDEM Collections

Learn how to query the CCMEO STAC API to discover available HRDEM mosaic collections.

In [2]:
from pystac_client import Client

# Connect to CCMEO STAC API
stac_api_url = "https://datacube.services.geo.ca/stac/api/"
catalog = Client.open(stac_api_url, modifier=None)

# Get all collections
collections = catalog.get_all_collections()

# Filter for HRDEM collections
hrdem_collections = [
    col for col in collections 
    if "hrdem" in col.id.lower() or "mrdem" in col.id.lower()
]

# Display collection information
for collection in hrdem_collections:
    print(f"Collection ID: {collection.id}")
    print(f"Description: {collection.description}")
    print(f"Extent: {collection.extent}")
    print()

Collection ID: hrdem-arcticdem
Description: High-Resolution Digital Elevation Model (HRDEM) generated from optical stereo imagery. This data collection includes a Digital Surface Model (DSM). The data in this collection have been reprojected from the source reference system to the Canada Atlas Lambert projection (EPSG:3979). / Modèle numérique d'élévation haute résolution (MNEHR) généré à partir de couple stéréo d'imagerie optique. Cette collection de données inclut un Modèle Numérique de Surface (MNS). Les données de cette collection ont été reprojetées du système de référence de la donnée source vers la projection Lambert de l'Atlas du Canada (EPSG:3979).
Extent: <pystac.collection.Extent object at 0x00000128DD9E5290>

Collection ID: hrdem-mosaic-2m
Description: The High Resolution Digital Elevation Model (HRDEM) Mosaic represents the current and continous coverage of high-resolution elevation data available in Canada. This version of the mosaic, available at a spatial resolution of 

### Output

This will show collections such as:
- `hrdem-mosaic-1m` - Mosaic of High Resolution Digital Elevation Model (HRDEM) at 1m
- `hrdem-mosaic-2m` - Mosaic of High Resolution Digital Elevation Model (HRDEM) at 2m
- `hrdem-lidar` - Mosaic of High Resolution Digital Elevation Model (HRDEM) by LiDAR acquisition project
- `hrdem-arcticdem` - Mosaic of High Resolution Digital Elevation Model (HRDEM) generated from optical stereo imagery (ArcticDEM)
- `mrdem-30` - Medium resolution digital elevation model - 30 meters (MRDEM-30)

Each collection includes metadata about its spatial extent, temporal coverage, and available bands.

## Example 2: Searching for Items by Area of Interest

Search for HRDEM tiles that intersect with a specific geographic area.

In [12]:
from pystac_client import Client
from shapely.geometry import box
import geopandas as gpd

# Define area of interest (example: around Ottawa)
aoi_bounds = (-75.85, 45.30, -75.60, 45.45)  # west, south, east, north
aoi_box = box(*aoi_bounds)

# Search STAC API for HRDEM 2m mosaic items
stac_api_url = "https://datacube.services.geo.ca/stac/api/"
catalog = Client.open(stac_api_url)

# Perform search
search = catalog.search(
    collections=["hrdem-mosaic-2m"],
    bbox=aoi_bounds,
    max_items=10
)

items = search.item_collection()

print(f"Found {len(list(items))} items intersecting AOI")

# Access asset URLs
for item in search.item_collection():
    print(f"\nItem ID: {item.id}")
    for asset_key, asset in item.assets.items():
        if "cloud-optimized" in asset.media_type:  # Cloud Optimized GeoTIFF asset
            print(f" {asset_key} COG URL: {asset.href}")
            cog_url = asset.href


Found 2 items intersecting AOI

Item ID: 8_2-mosaic-2m
 dsm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/8_2-mosaic-2m-dsm.tif
 dtm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/8_2-mosaic-2m-dtm.tif
 hillshade-dsm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/8_2-mosaic-2m-dsm_hillshade.tif
 hillshade-dtm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/8_2-mosaic-2m-dtm_hillshade.tif

Item ID: 9_2-mosaic-2m
 dsm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dsm.tif
 dtm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dtm.tif
 hillshade-dsm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dsm_hillshade.tif
 hillshade-dtm COG URL: https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dtm

### Key Parameters

- `collections`: Specify which STAC collection to search (e.g., "hrdem-mosaic-2m")
- `bbox`: Bounding box in [west, south, east, north] format
- `max_items`: Limit the number of results returned

## Example 3: Accessing and Processing COG Data

Load and process HRDEM data from the remote COG files.

### Using rasterio (Low-level Access)

In [20]:
import rasterio
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds
import numpy as np

for item in search.item_collection():
    for asset_key, asset in item.assets.items():
        # We use 9_2 because it covers our AOI
        if "9_2" in item.id and  asset_key == "dtm" and "cloud-optimized" in asset.media_type:  # Cloud Optimized GeoTIFF asset
            cog_url = asset.href
# Uncomment/Replace the next line if you want to use a specific URL
# cog_url = "https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dtm.tif"

# Transform AOI bounds from geographic (EPSG:4617) to LCC (EPSG:3979)
aoi_bounds_transformed = transform_bounds(
    "EPSG:4617",  # Source CRS (NAD83 geographic)
    "EPSG:3979",  # Target CRS (the COG's CRS, EPSG:3979)
    *aoi_bounds
)
print(f"AOI bounds transformed to EPSG:3979: {aoi_bounds_transformed}")

# Open remote COG
with rasterio.open(cog_url) as src:
    # Read full metadata
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")
    print(f"Bounds: {src.bounds}")
    
    # Read data in a specific window (efficient for large files)
    window = from_bounds(*aoi_bounds_transformed, src.transform)
    data = src.read(1, window=window)  # Read first band (elevation)
    
    # Get statistics
    if data.size > 0:
        print(f"Min elevation: {np.nanmin(data):.2f} m")
        print(f"Max elevation: {np.nanmax(data):.2f} m")
        print(f"Mean elevation: {np.nanmean(data):.2f} m")
    else:
        print("No data available in the selected window.")

AOI bounds transformed to EPSG:3979: (1498068.328271543, -186524.45076723737, 1522103.6826346598, -164429.1884248994)
CRS: EPSG:3979
Resolution: (2.0, 2.0)
Bounds: BoundingBox(left=1500000.0, bottom=-500000.0, right=2000000.0, top=0.0)
Min elevation: 40.64 m
Max elevation: 219.07 m
Mean elevation: 83.25 m


### Using rioxarray (Array-based Access)

In [21]:
import rioxarray
import rasterio.crs

# Open remote COG as xarray DataArray
# Uncomment/Replace the next line if you want to use a specific URL
# cog_url = "https://canelevation-dem.s3.ca-central-1.amazonaws.com/hrdem-mosaic-2m/9_2-mosaic-2m-dtm.tif"
elevation = rioxarray.open_rasterio(cog_url)

print(f"Data shape: {elevation.shape}")
print(f"Coordinates: {elevation.coords}")

# Clip to area of interest
clipped = elevation.rio.clip_box(*aoi_bounds_transformed)

# Generate statistics
stats = {
    "min": float(clipped.min()),
    "max": float(clipped.max()),
    "mean": float(clipped.mean()),
    "std": float(clipped.std())
}
print(f"Elevation statistics: {stats}")

# Export to file
clipped.rio.to_raster("hrdem_subset.tif")

Data shape: (1, 250000, 250000)
Coordinates: Coordinates:
  * band         (band) int64 8B 1
  * y            (y) float64 2MB -1.0 -3.0 -5.0 -7.0 ... -5e+05 -5e+05 -5e+05
  * x            (x) float64 2MB 1.5e+06 1.5e+06 1.5e+06 ... 2e+06 2e+06 2e+06
    spatial_ref  int64 8B 0
Elevation statistics: {'min': 40.63631820678711, 'max': 219.06988525390625, 'mean': 83.24858093261719, 'std': 19.521799087524414}
